In [1]:
import os
import sys
sys.path.append('..')
os.environ['PATH'] = f"{os.path.dirname(sys.executable)}:{os.environ['PATH']}"

In [2]:
from src.utils import get_TFBS_from_ATAC, get_TFBS_from_promoter, build_Graph, build_cross_graph, extract_atac_feature, extract_scrna_feature2

In [3]:
cell_type = "B_naive"
comment = "test"

In [4]:
atac_file = os.path.join(cell_type, 'atac', 'matrix.mtx')
scrna_file = os.path.join(cell_type, 'scrna', 'matrix.mtx')
save_dir = os.path.join("graph", cell_type)

In [5]:
motif_dir = "../data_resource/HOCOMOCOv11"
genome_file = "../data_resource/download_genome/hg38"
promoter_file = "../data_resource/gencode.v38.ProteinCoding_gene_promoter.txt"

In [6]:
atac_region_file = get_TFBS_from_ATAC(atac_file, motif_dir, genome_file, overwrite=False, save_dir=save_dir)
atac_region_file

'graph/B_naive/region/atac_tfbs_region_all_tmp.txt'

In [7]:
promoter_region_file = get_TFBS_from_promoter(promoter_file, motif_dir, overwrite=False, save_dir=save_dir)
promoter_region_file

'graph/B_naive/region/gencode.TF.promoter.region.txt'

In [8]:
graph_file, node_file = build_Graph(atac_region_file, promoter_region_file, overwrite=False, save_dir=save_dir)

In [9]:
atac_feature_file = extract_atac_feature(atac_file, node_file, cell_num=None, seed=666, overwrite=False, save_dir=save_dir)
atac_feature_file

'graph/B_naive/origin_graph/atac_rp_score_feature.txt'

In [10]:
scrna_feature_file = extract_scrna_feature2(scrna_file, node_file, cell_num=None, seed=666, overwrite=False, just_node=False, save_dir=save_dir)
scrna_feature_file

'graph/B_naive/origin_graph/scRNA_grnboost2_feature.txt'

In [11]:
scrna_tmp_file = os.path.join(save_dir, 'tmp', 'scrna.csv')

In [12]:
cross_graph_file, cross_node_file, cross_atac_file, aross_scrna_file = \
   build_cross_graph(graph_file, atac_feature_file, scrna_feature_file, overwrite=True, save_dir=save_dir)

final graph with 87 nodes, 946 edges, density: 0.12498348526885983


In [13]:
from src.trainer import Trainer

In [14]:
parser = Trainer.add_argparse_args(name=cell_type, 
                                   adj_file=cross_graph_file, 
                                   node_file=cross_node_file, 
                                   scrna_file=scrna_tmp_file,
                                   comment=comment,
                                   atac_feature_file=cross_atac_file, 
                                   scrna_feature_file=aross_scrna_file,
                                   model_cls='scMultiomeGRN', 
                                   mask_rate=0.0, 
                                   data_dir='tmp',
                                   )
config = parser.parse_args([])
config.accelerator = "auto"

In [15]:
config

Namespace(name='B_naive', adj_file='graph/B_naive/x_graph.txt', node_file='graph/B_naive/x_node.txt', atac_feature_file='graph/B_naive/x_atac_rp_score_feature.txt', scrna_feature_file='graph/B_naive/x_scRNA_grnboost2_feature.txt', scrna_file='graph/B_naive/tmp/scrna.csv', dataset_seed=666, train_split_rate=0.8, n_splits=10, data_dir='tmp', split_id=1, max_epochs=2000, patience=100, model_cls='scMultiomeGRN', model_seed=666, comment='test', accelerator='auto', mask_rate=0.0)

In [16]:
Trainer.run_single_fold(**vars(config))

skip split_dataset with seed:666, train_split_rate:0.8, n_splits:10
load from tmp/B_naive/seed-666_fold-10_rate-0.8_all-train-True/split-001_train-edge-338_87x87.csv
adj:87x87, edge:762, atac:87x382, scrna:87x2069, edge_attr:torch.Size([763, 16, 16]), pos_weight:10.19675, norm_weight:0.54904


/tmp/pycharm_project_732/demo/../src/dataset.py:42: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:641.)
  return torch.sparse.FloatTensor(indices, values, shape)
Seed set to 666


load from tmp/B_naive/seed-666_fold-10_rate-0.8_all-train-True/split-001_val-edge-85_87x87.csv
adj:87x87, edge:171, atac:87x382, scrna:87x2069, edge_attr:torch.Size([257, 16, 16]), pos_weight:43.52353, norm_weight:0.51149
load from tmp/B_naive/seed-666_fold-10_rate-0.8_all-train-True/split-001_test-edge-47_87x87.csv
adj:87x87, edge:94, atac:87x382, scrna:87x2069, edge_attr:torch.Size([181, 16, 16]), pos_weight:79.52128, norm_weight:0.50629


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/lcc/miniconda3/envs/scMultiomeGRN/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA GeForce RTX 4090 D') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#

Sanity Checking DataLoader 0:   0%|                                                                           | 0/1 [00:00<?, ?it/s]

...

Restoring states from the checkpoint path at lightning_logs/B_naive/scMultiomeGRN/test/split_1/lightning_logs/version_3/checkpoints/epoch=44-step=45.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at lightning_logs/B_naive/scMultiomeGRN/test/split_1/lightning_logs/version_3/checkpoints/epoch=44-step=45.ckpt
/home/lcc/miniconda3/envs/scMultiomeGRN/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=18` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|███████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 64.46it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test/best_test/acc       0.7446808218955994
    test/best_test/ap       0.8161128163337708
   test/best_test/aupr      0.8137375116348267
  test/best_test/auroc      0.8107740879058838
   test/best_test/loss      1.4928151278127075
 test/best_test/loss_rec    1.4928151278127075
test/best_test/precision    0.7446808218955994
  test/best_test/recall     0.7446808218955994
test/best_test/threshold    0.5835816860198975
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Restoring states from the checkpoint path at lightning_logs/B_naive/scMultiomeGRN/test/split_1/lightning_logs/version_3/checkpoints/epoch=44-step=45.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at lightning_logs/B_naive/scMultiomeGRN/test/split_1/lightning_logs/version_3/checkpoints/epoch=44-step=45.ckpt
/home/lcc/miniconda3/envs/scMultiomeGRN/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=18` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|███████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  8.17it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
    test/best_all/acc       0.7624521255493164
    test/best_all/ap        0.47304728627204895
   test/best_all/aupr       0.47085925936698914
   test/best_all/auroc      0.8355786800384521
   test/best_all/loss       0.7885730070194796
 test/best_all/loss_rec     0.7885730070194796
 test/best_all/precision    0.3142857253551483
  test/best_all/recall      0.7723404169082642
 test/best_all/threshold    0.5835816860198975
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
lightning_logs/B_naive/scMultiomeGR

'lightning_logs/B_naive/scMultiomeGRN/test/split_1/lightning_logs/version_3'

In [ ]:
# Trainer.run_all_fold(**vars(config))